In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy.linalg import solve_triangular
from scipy.special import kv

sys.path.append('/Users/xiangshi/Documents/GitHub/normix/')

from normix.gig import GIG
from normix.func import logkv, kvratio

%matplotlib inline

In [139]:
class GH(object):
    """
    Generalized Hyperbolic (GH)
    """

    def __init__(self, mu, gamma, sigma, lam, chi, psi):
        
        self.mu = mu
        self.gamma = gamma
        self.sigma = sigma
        self.lam = lam
        self.chi = chi
        self.psi = psi
        self.dim = len(mu)
        
    def pdf(self, x):
        l = np.linalg.cholesky(self.sigma)
        z = solve_triangular(l, (x-self.mu).T, lower=True)
        gamma = solve_triangular(l, self.gamma)
        lam = self.lam-self.dim/2
        chi = self.chi+np.sum(z**2, axis=0)
        psi = self.psi+np.dot(gamma, gamma)

        c = (self.psi/self.chi)**(self.lam/2)/(2*np.pi)**(self.dim/2)/ \
            np.prod(np.diag(l))/kv(self.lam, np.sqrt(self.chi*self.psi) + 0.0j).real
        p = c*kv(lam, np.sqrt(chi*psi))*np.exp(np.dot(z.T, gamma))* \
            np.sqrt(psi/chi)**(self.dim/2-self.lam);
        return p
    
    def llh(self, x):
        l = np.linalg.cholesky(self.sigma)
        z = solve_triangular(l, (x-self.mu).T, lower=True)
        gamma = solve_triangular(l, self.gamma)
        lam = self.lam-self.dim/2
        chi = self.chi+np.sum(z**2, axis=0)
        psi = self.psi+np.dot(gamma, gamma)
        
        delta = np.sqrt(chi/psi)
        eta = np.sqrt(chi*psi)
        
        llh = self.lam/2*np.log(self.psi/self.chi) \
            - logkv(self.lam, np.sqrt(self.chi*self.psi)) \
            + logkv(lam, eta) + np.dot(z.T, gamma) + lam*np.log(delta)
        return llh, lam, delta, eta
        
    
    def mean(self):
        return self.mu+self.gamma*GIG(self.lam, self.chi, self.psi).mean()
    
    def cov(self):
        gig = GIG(self.lam, self.chi, self.psi)
        return self.sigma*gig.mean() + np.outer(self.gamma, self.gamma)*gig.var()
    
    def fit_em(self, x, max_iter=100, fix_tail=False, eps=1e-5, reg='|sigma|=1', diff=1e-5, disp=True):
        if self.dim != x.shape[1]:
            raise ValueError('x dimension must be (, {})'.format(self.dim))
            
        suff_stats = [0]*6
        suff_stats[3] = np.mean(x, axis=0)
        
        llh_last = 0
        for i in range(max_iter):
            # self.regulate(reg)
            llh, lam, delta, eta = self.llh(x)
                
            if i>0:
                if disp:
                    print('iter=%s, llh=%.5f, change=%.5f'%(i, llh, llh-llh_last))
                
                if np.abs(llh-llh_last)<eps:
                    if disp:
                        print('success')
                    return True
            llh_last = llh
        
            # E-step
            a = kvratio(lam-1, lam, eta)/delta
            b = kvratio(lam+1, lam, eta)*delta
            c = (kvratio(lam+diff, lam, eta)-kvratio(lam-diff, lam, eta))/(2*diff)+np.log(delta)

            suff_stats[0] = np.mean(a)
            suff_stats[1] = np.mean(b)
            suff_stats[2] = np.mean(c)
            suff_stats[4] = np.mean(x.T*a, axis=1)
            suff_stats[5] = np.dot((x.T*a)/x.shape[0], x)
            
            if not fix_tail:
                gig = GIG(gh.lam, gh.chi, gh.psi)
                res = gig.suffstats2param(suff_stats[0], suff_stats[1], suff_stats[2])
                if res.success:
                    self.lam = gig.lam
                    self.chi = gig.chi
                    self.psi = gig.psi
                
            self.mu = (suff_stats[3]-suff_stats[1]*suff_stats[4])/(1-suff_stats[0]*suff_stats[1])
            self.gamma = (suff_stats[4]-suff_stats[0]*suff_stats[3])/(1-suff_stats[0]*suff_stats[1])
            self.sigma = -np.outer(suff_stats[4], self.mu)
            self.sigma = self.sigma + self.sigma.T + suff_stats[5] \
                + suff_stats[0]*np.outer(self.mu, self.mu) \
                - suff_stats[1]*np.outer(self.gamma, self.gamma)
            # self.sigma = np.maximum(self.sigma, self.sigma.T)
            
        if disp:
            print('fail to converge')
        return False
    
    
    def rvs(self, n, n_grids=int(1e5)):
        t = GIG(self.lam, self.chi, self.psi).rvs(n, n_grids)
        z = np.random.multivariate_normal(np.zeros(self.dim), self.sigma, n)
        x = (np.outer(self.gamma, t) + z.T*np.sqrt(t)).T + mu
        return x
    
    
    def regulate(self, method='|sigma|=1'):
        """ GH parameterization (mu, gamma Sigma, lambda, chi, psi) is not unique, so regulation is needed.
        """
        if method=='chi=1':
            self.gamma = self.gamma*self.chi
            self.sigma = self.sigma*self.chi
            self.psi = self.psi*self.chi
            self.chi = 1.0
        elif method=='psi=1':
            self.gamma = self.gamma/self.psi
            self.sigma = self.sigma/self.psi
            self.chi = self.chi*self.psi
            self.psi = 1.0
        elif method=='chi=psi':
            self.gamma = self.gamma*np.sqrt(self.chi/self.psi)
            self.sigma = self.sigma*np.sqrt(self.chi/self.psi)
            self.chi = np.sqrt(self.chi*self.psi)
            self.psi = self.chi
        elif method=='|sigma|=1':
            _, logd = np.linalg.slogdet(self.sigma)
            d = np.exp(logd/self.dim)
            self.gamma = self.gamma/d
            self.sigma = self.sigma/d
            self.chi = self.chi*d
            self.psi = self.psi/d


In [110]:
mu = np.random.randn(10)
sigma = np.cov(np.random.randn(10, 20))
gamma = np.random.randn(10)
lam=0.9
chi=0.3
psi=1.0

In [111]:
mu = np.array([0.65634476,  1.51842634, -0.63371085])
sigma= np.array([[0.35646818, 0.10212985, 0.05799233],
       [0.10212985, 0.98229971, 0.15375874],
       [0.05799233, 0.15375874, 1.41077451]])
gamma = np.array([ 2.05705176, -0.04197259,  0.34341511])
lam=0.9
chi=0.3
psi=1.0

mu0 = np.zeros(3)
sigma0 = np.eye(3)
gamma0 = np.zeros(3)
lam0 = 1.0
chi0 = 1.0
psi0 = 1.0

In [112]:
gh = GH(mu, gamma, sigma, lam, chi, psi)
m = gh.mean()
c = gh.cov()
for method in ['|sigma|=1', 'chi=1', 'psi=1', 'chi=psi']:
    gh = GH(mu, gamma, sigma, lam, chi, psi)
    gh.regulate(method)
    print(np.linalg.norm(gh.mean()-m)/np.linalg.norm(m), 
          np.linalg.norm(gh.cov()-c)/np.linalg.norm(c))

1.6949953239248218e-16 4.096647724127068e-16
2.102382951329647e-17 2.0234082797393442e-16
0.0 0.0
2.102382951329647e-17 3.54598817343108e-17


In [113]:
gh = GH(mu, gamma, sigma, lam, chi, psi)
gh.regulate()
x = gh.rvs(1000)
print(np.linalg.norm(np.mean(x, axis=0)-m)/np.linalg.norm(m), 
      np.linalg.norm(np.cov(x.T)-c)/np.linalg.norm(c))

0.027160269672164564 0.07101347388016566


In [114]:
x = np.array([[ 1.74953772,  1.37746568, -1.54835839],
       [ 3.76765938,  1.54792901, -0.36846954],
       [ 5.70719392,  0.82978642, -2.07754969],
       [ 1.95051383,  0.62586649, -0.66932423],
       [ 7.93513039,  0.54582224,  0.90509363],
       [11.64517463, -0.26257948,  7.62055921],
       [ 0.47822655,  2.261273  , -0.68519925],
       [14.36924369,  2.9331593 , -0.39317326],
       [ 3.06647501,  0.94545944,  0.87562929],
       [ 2.36628983,  1.99396013, -0.60114535]])



In [115]:
gh_fit = GH(mu0, gamma0, sigma0, lam0, chi0, psi0)

In [135]:
diff = 1e-5

In [136]:
suff_stats = [0]*6
suff_stats[3] = np.mean(x, axis=0)
llh_last = 0
i = 0

In [142]:
llh, lam, delta, eta = gh_fit.llh(x)

# E-step
a = kvratio(lam-1, lam, eta)/delta
b = kvratio(lam+1, lam, eta)*delta
c = (kvratio(lam+diff, lam, eta)-kvratio(lam-diff, lam, eta))/(2*diff)+np.log(delta)

suff_stats[0] = np.mean(a)
suff_stats[1] = np.mean(b)
suff_stats[2] = np.mean(c)
suff_stats[4] = np.mean(x.T*a, axis=1)
suff_stats[5] = np.dot((x.T*a)/x.shape[0], x)

In [145]:
gig = GIG(gh.lam, gh.chi, gh.psi)
res = gig.suffstats2param(suff_stats[0], suff_stats[1], suff_stats[2])

/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


In [147]:
res.x

array([0.0918306 , 4.65295278, 0.26519496])

In [148]:
self.mu = (suff_stats[3]-suff_stats[1]*suff_stats[4])/(1-suff_stats[0]*suff_stats[1])
self.gamma = (suff_stats[4]-suff_stats[0]*suff_stats[3])/(1-suff_stats[0]*suff_stats[1])
self.sigma = -np.outer(suff_stats[4], gh_fit.mu)
gh_fit.sigma = gh_fit.sigma + gh_fit.sigma.T + suff_stats[5] \
    + suff_stats[0]*np.outer(gh_fit.mu, gh_fit.mu) \
    - suff_stats[1]*np.outer(gh_fit.gamma, gh_fit.gamma)

NameError: name 'self' is not defined

In [138]:
(kvratio(lam+diff, lam, eta)-kvratio(lam-diff, lam, eta))

array([-3.00213049e-06, -2.14346395e-06, -1.49743216e-06, -3.56217451e-06,
       -1.17081863e-06, -6.92563171e-07, -3.28326966e-06, -6.58354477e-07,
       -2.55025381e-06, -2.66455083e-06])

In [134]:
eta

array([ 2.89062409,  4.21035761,  6.21102412,  2.37575419,  8.067429  ,
       13.95535606,  2.60989544, 14.70486902,  3.47331098,  3.30705009])

In [121]:
kv(lam-1, eta)/kv(lam, eta)

array([1.34594605, 1.23750952, 1.16100404, 1.42091897, 1.12395523,
       1.07165708, 1.38315711, 1.06800469, 1.28790972, 1.30238429])

In [69]:
gh_fit.fit_em(x, fix_tail=False)

iter=1, llh=-4.07511, change=5.72913
iter=2, llh=-0.59984, change=3.47526
iter=3, llh=-2.11217, change=-1.51233
iter=4, llh=-2.57087, change=-0.45870
iter=5, llh=-2.87619, change=-0.30532
iter=6, llh=-3.00005, change=-0.12386


/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


iter=7, llh=-3.15745, change=-0.15740
iter=8, llh=-3.19400, change=-0.03656
iter=9, llh=-3.29609, change=-0.10209
iter=10, llh=-3.34145, change=-0.04536
iter=11, llh=-3.39780, change=-0.05635


/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


iter=12, llh=-3.42807, change=-0.03026
iter=13, llh=-3.48110, change=-0.05303
iter=14, llh=-3.49623, change=-0.01514


...

iter=32, llh=-3.82195, change=-0.01609


/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)
/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)
/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can

iter=33, llh=-3.82992, change=-0.00796


/Users/xiangshi/anaconda3/envs/deep_learning_course/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


KeyboardInterrupt: 

In [ ]:
from scipy.special import factorial
k = np.arange(1, 10)

In [ ]:
factorial(k)

In [ ]:
k

In [ ]:
gh_fit.sigma

In [ ]:

diff = 1e-8
reg = '|sigma|=1'

suff_stats = [0]*6
suff_stats[3] = np.mean(x, axis=0)

gh.regulate(reg)

In [ ]:
llh, lam, delta, eta = gh.llh(x)

In [ ]:
s = kvratio(lam-1, lam, eta)

suff_stats[0] = np.mean(s)
suff_stats[1] = np.mean(kvratio(lam+1, lam, eta))
suff_stats[2] = np.mean((kvratio(lam+diff, lam, eta)-kvratio(lam-diff, lam, eta))/(2*diff))
suff_stats[4] = np.mean(x.T*s, axis=1)
suff_stats[5] = np.dot((x.T*s)/x.shape[0], x)
suff_stats[5] = np.maximum(suff_stats[5], suff_stats[5].T)

In [ ]:
mu = (S4-S2*S5)/(1-S1*S2);
    gamma = (S5-S1*S4)/(1-S1*S2);
    Sigma = S6-S5*mu'-mu*S5'+S1*(mu*mu')-S2*(gamma*gamma');

In [149]:
gh_fit.mu = (suff_stats[3]-suff_stats[1]*suff_stats[4])/(1-suff_stats[0]*suff_stats[1])
gh_fit.gamma = (suff_stats[4]-suff_stats[0]*suff_stats[3])/(1-suff_stats[0]*suff_stats[1])
gh_fit.sigma = -np.outer(suff_stats[4], gh_fit.mu)
gh_fit.sigma = gh_fit.sigma + gh_fit.sigma.T + suff_stats[5] \
    + suff_stats[0]*np.outer(gh_fit.mu, gh_fit.mu) \
    - suff_stats[1]*np.outer(gh_fit.gamma, gh_fit.gamma)
# gh_fit.sigma[5] = np.maximum(gh_fit.sigma[5], gh_fit.sigma[5].T)

In [151]:
gh_fit.mu

array([ 0.62270148,  1.42936867, -1.12930602])

In [152]:
gh_fit.sigma

array([[ 0.78617701, -0.09383308,  0.14378663],
       [-0.09383308,  0.14945062, -0.10842705],
       [ 0.14378663, -0.10842705,  0.54143216]])

In [153]:
gh_fit.gamma

array([ 0.75734847, -0.02419753,  0.23219751])

In [ ]:
gig = GIG(gh.lam, gh.chi, gh.psi)
res = gig.suffstats2param(suff_stats[0], suff_stats[1], suff_stats[2])

In [ ]:
res.success

In [ ]:
gh_update = GH(self.mu, self.gamma, self.sigma, gig.lam, gig.chi, gig.psi)

In [ ]:
gh_update.regulate()

In [ ]:
gh_update.llh(x)

In [ ]:
gh.llh(x)

In [ ]:
gh_update.sigma.shape

In [ ]:
np.linalg.det(gh_update.sigma)

In [ ]:
np.linalg.svd(gh_update.sigma)

In [ ]:
for i in range(10):
    gh_univ = GH(mu=gh.mu[[i]], 
                 sigma=np.array([[np.diag(gh.sigma)[i]]]), 
                 gamma=gh.gamma[[i]], 
                 lam=gh.lam, chi=gh.chi, psi=gh.psi)
    nbins = 1000
    hist, grids = np.histogram(x[:, i], bins=nbins)
    dx = grids[1]-grids[0]

    xi = (grids[1:]-dx/2).reshape((nbins, 1))
    pdf = gh_univ.pdf(xi)*dx

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.step(grids[1:], hist/np.sum(hist))
    ax.step(xi, pdf)

In [ ]:
np.mean(x, axis=0).shape

In [ ]:
'iter=%s, llh=%.2f, change='%(1, llh, chg)